<a href="https://colab.research.google.com/github/jalane76/adversarial-attacks-tutorial/blob/main/adversarial_attacks_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 7.6 MB/s 
     |████████████████████████████████| 3.4 MB 43.5 MB/s 
     |████████████████████████████████| 25.3 MB 85 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [3]:
from art.utils import load_mnist
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
# Set up
rand_seed = 978614566
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

image_width = 28
image_height = 28
input_shape = image_width * image_height

batch_size = 64
num_epochs = 15
num_labels = 10

In [7]:
 # Load data
(
  (x_train, y_train),
  (x_test, y_test),
  min_pixel_value,
  max_pixel_value,
) = load_mnist()
clip_values = {
  "min_pixel_value": min_pixel_value,
  "max_pixel_value": max_pixel_value,
}

 # Swap axes to PyTorch's NCHW format
x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

In [ ]:
# Define model
class Mnist3LayerLinearModel(nn.Module):
  def __init__(self):
    super(Mnist3LayerLinearModel, self).__init__()
    self.lin_1 = nn.Linear(in_features=784, out_features=128)
    self.lin_2 = nn.Linear(in_features=128, out_features=64)
    self.lin_3 = nn.Linear(in_features=64, out_features=10)

    self.criterion = nn.NLLLoss()
    self.optimizer = optim.SGD(self.parameters(), lr=0.003, momentum=0.9) 

  def forward(self, x):
    x = F.relu(self.lin_1(x))
    x = F.relu(self.lin_2(x))
    x = F.log_softmax(self.lin_3(x), dim=1)
    return x  

# Make ART classifier
classifier = PyTorchClassifier(
  model=model,
  clip_values=clip_values,
  loss=model.criterion,
  optimizer=model.optimizer,
  input_shape=(input_shape),
  nb_classes=num_labels,
)

# Train classifier
  classifier.fit(x, y, batch_size=config.batch_size, nb_epochs=config.num_epochs)